# Toxic comments detection ' JigSaw' Using LSTM 
Data can be found on Kaggle. 

In [ ]:
 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.preprocessing import text, sequence
from keras import backend as K
from keras.models import load_model
import keras
import pickle
print(K.tensorflow_backend._get_available_gpus())
 
import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
 

### Preprocessing Data

In [ ]:
train_data = train["comment_text"] # Select the comment attribute column
label_data = train["target"] # Labels 
test_data = test["comment_text"]
train_data.shape, label_data.shape, test_data.shape

#### Vectorize Data

In [ ]:
tokenizer = text.Tokenizer()
tokenizer.fit_on_texts(list(train_data) + list(test_data))

In [ ]:
train_data = tokenizer.texts_to_sequences(train_data)
test_data = tokenizer.texts_to_sequences(test_data)

In [ ]:
MAX_LEN = 200
train_data = sequence.pad_sequences(train_data, maxlen=MAX_LEN)
test_data = sequence.pad_sequences(test_data, maxlen=MAX_LEN)

In [ ]:
max_features = None

In [ ]:
max_features = max_features or len(tokenizer.word_index) + 1
max_features

In [ ]:
type(train_data), type(label_data.values), type(test_data)
label_data = label_data.values

#### Model

In [ ]:
# Model Parameters
NUM_HIDDEN = 256
EMB_SIZE = 256
LABEL_SIZE = 1
MAX_FEATURES = max_features
DROP_OUT_RATE = 0.2
NUM_EPOCH = 10

# Optimization Parameters
BATCH_SIZE = 1000
LOSS_FUNC = "binary_crossentropy"
OPTIMIZER_FUNC = "adam"
METRICS = ["accuracy"]

class LSTMModel:
    
    def __init__(self):
        self.model = self.build_graph()
        self.compile_model()
    
    def build_graph(self):
        model = keras.models.Sequential([
            keras.layers.Embedding(MAX_FEATURES, EMB_SIZE),
            keras.layers.CuDNNLSTM(NUM_HIDDEN),
            keras.layers.Dropout(rate=DROP_OUT_RATE),
            keras.layers.Dense(LABEL_SIZE, activation='sigmoid')])
        return model
    
    def compile_model(self):
        self.model.compile(
            loss=LOSS_FUNC,
            optimizer=OPTIMIZER_FUNC,
            metrics=METRICS)

In [ ]:
model = LSTMModel().model
model.fit(
    train_data, 
    label_data, 
    batch_size = BATCH_SIZE, 
    epochs = NUM_EPOCH)

#### Prediction

In [ ]:
submission_in = 'sample_submission.csv'
submission_out = 'submission.csv'

In [ ]:
result = model.predict(test_data)

In [ ]:
submission = pd.read_csv(submission_in, index_col='id')
submission['prediction'] = result
submission.reset_index(drop=False, inplace=True)